In [1]:
import os  
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2 pyspark-shell'  
from pyspark import SparkContext  
from pyspark.streaming import StreamingContext  
from pyspark.streaming.kafka import KafkaUtils   

batch_interval = 1
window_length = 12 * batch_interval
frequency = 6 * batch_interval

sc = SparkContext(appName="CTR")
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, batch_interval)

# ksClick = KafkaUtils.createDirectStream(ssc, ['topic_click'], {"metadata.broker.list": 'ec2-52-9-120-149.us-west-1.compute.amazonaws.com:9092'})
ksView = KafkaUtils.createDirectStream(ssc, ['topic_view'], {"metadata.broker.list": 'ec2-52-9-120-149.us-west-1.compute.amazonaws.com:9092'})

In [2]:
# linesClick = ksClick.map(lambda x: x[1])
linesView = ksView.map(lambda x: x[1])

In [3]:
import re
def mk_int(s):
    s = s.strip()
    return int(s) if s else 0
def tuple_sum(a, b):
    return [a[0]+b[0], a[1]+b[1]]
def tuple_div(a, b):
    return [a[0]/b[0], a[1]/b[1]]

In [4]:
linesView.pprint(10)

In [5]:
# countClick = linesClick.flatMap(lambda line: [line.split("\t")[-1]]) \
#     .map(lambda click: ("ctr", [mk_int(click), 1])) \
#     .reduceByKeyAndWindow(lambda a, b: tuple_sum(a, b), None, 12, 6)

# countView = linesView.flatMap(lambda line: [line.split("\t")[-1]]) \
#     .map(lambda click: ("ctr", [mk_int(click), 1])) \
#     .reduceByKeyAndWindow(lambda a, b: tuple_sum(a, b), None, 12, 6) 
    
#test = countClick.join(countView)
#test.pprint(20)

# countClick.pprint(10)
# countView.pprint(10)

ssc.start()  
ssc.awaitTermination()

-------------------------------------------
Time: 2017-09-19 23:46:34
-------------------------------------------
2545367	2015-05-10 15:15:15.0	457149	3611833	0		2840	27	

2545368	2015-05-09 20:00:06.0	1736694	2925312	1		2643	1	

2545369	2015-05-01 09:37:51.0	1876996	1327591	1		1294	3	

2545370	2015-05-08 13:39:05.0	2165179	224813	0		3823	22	{83:'Свадебные платья', 175:'Женская одежда'}

2545371	2015-04-25 18:25:33.0	1923006	1839867	0		1309	34	{5:'Аксессуары'}

2545372	2015-05-10 22:17:13.0	1406160	3914976	0		3953	38	

2545373	2015-05-10 08:13:50.0	2139208	2513610	0		1102	34	{5:'Аудио- и видеотехника'}

2545374	2015-05-04 09:54:37.0	1293929	2682179	0		3960	60	{115:'134-140 см (8-10 лет)', 179:'Платья и юбки', 178:'Для девочек'}

2545375	2015-05-13 18:55:33.0	731406	1610643	0		2885	26	{132:'Усилители и ресиверы'}

2545376	2015-05-05 20:29:27.0	771196	2315845	1		2610	44	

...



Py4JJavaError: An error occurred while calling o28.awaitTermination.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/usr/local/spark/python/pyspark/streaming/util.py", line 65, in call
    r = self.func(t, *rdds)
  File "/usr/local/spark/python/pyspark/streaming/dstream.py", line 171, in takeAndPrint
    taken = rdd.take(num + 1)
  File "/usr/local/spark/python/pyspark/rdd.py", line 1342, in take
    res = self.context.runJob(self, takeUpToNumLeft, p)
  File "/usr/local/spark/python/pyspark/context.py", line 968, in runJob
    port = self._jvm.PythonRDD.runJob(self._jsc.sc(), mappedRDD._jrdd, partitions)
  File "/usr/local/lib/python2.7/dist-packages/py4j/java_gateway.py", line 1160, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/local/lib/python2.7/dist-packages/py4j/protocol.py", line 320, in get_return_value
    format(target_id, ".", name), value)
Py4JJavaError: An error occurred

-------------------------------------------
Time: 2017-09-19 23:46:36
-------------------------------------------
2551586	2015-05-15 22:01:53.0	1735498	554830	0		346	12	

2551587	2015-05-19 18:02:38.0	1684447	268692	0	диван бу	4464	38	{45:'Кровати, диваны и кресла'}

2551588	2015-04-26 23:59:00.0	833343	557630	0		1110	26	{132:'Видеокамеры'}

2551589	2015-04-30 17:00:33.0	135251	1345758	0		1261	22	

2551590	2015-05-01 00:14:21.0	1723963	858910	0		4172	46	

2551591	2015-05-11 14:18:17.0	1513478	3135185	0		733	60	{178:'Для девочек'}

2551592	2015-05-16 23:21:39.0	2040922	1505925	0		1261	26	{132:'Телевизоры и проекторы'}

2551593	2015-05-10 06:23:04.0	359484	1112339	1		1344	38	{45:'Другое'}

2551594	2015-05-13 22:14:36.0	2170395	931018	0		565	34	{5:'Шины, диски и колёса'}

2551595	2015-05-05 15:08:27.0	729965	2935347	0		3960	1	

...

-------------------------------------------
Time: 2017-09-19 23:46:37
-------------------------------------------
2555408	2015-05-07 14:59:59.0	1447838	233549

In [ ]:
ssc.stop()
sc.stop()